## plot time series of hindcast and forecast at lead year ii

In [1]:
import xarray as xr
import netCDF4
import numpy as np
from scipy.stats import pearsonr
import os

In [2]:
USER = os.environ['USER']
datadir = f'/glade/scratch/{USER}/DPLE-results'
var = 'Phi'
tres = 'yearly'

## FOSI hindcast 

In [3]:
filename = f'{datadir}/DPLE-FOSI_hindcast_{var}_Eo_space_LME_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:          (lme: 11, time: 70)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * time             (time) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Data variables: (12/39)
    Phi_0_0-200m     (lme, time) float64 ...
    Phi_0_200-600m   (lme, time) float64 ...
    Phi_0_0-600m     (lme, time) float64 ...
    Phi_1_0-200m     (lme, time) float64 ...
    Phi_1_200-600m   (lme, time) float64 ...
    Phi_1_0-600m     (lme, time) float64 ...
    ...               ...
    Phi_11_0-200m    (lme, time) float64 ...
    Phi_11_200-600m  (lme, time) float64 ...
    Phi_11_0-600m    (lme, time) float64 ...
    Phi_12_0-200m    (lme, time) float64 ...
    Phi_12_200-600m  (lme, time) float64 ...
    Phi_12_0-600m    (lme, time) float64 ...

## DPLE forecast

In [4]:
filename = f'{datadir}/DPLE_driftcorrected_Phi_Eo_space_LME_ens_mean_{tres}.nc'
ds_fore = xr.open_dataset(filename)
ds_fore

<xarray.Dataset>
Dimensions:          (lme: 11, Y: 64, L: 10)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y                (Y) int64 1954 1955 1956 1957 1958 ... 2014 2015 2016 2017
  * L                (L) int64 0 1 2 3 4 5 6 7 8 9
Data variables: (12/39)
    Phi_0_0-200m     (lme, Y, L) float64 ...
    Phi_0_200-600m   (lme, Y, L) float64 ...
    Phi_0_0-600m     (lme, Y, L) float64 ...
    Phi_1_0-200m     (lme, Y, L) float64 ...
    Phi_1_200-600m   (lme, Y, L) float64 ...
    Phi_1_0-600m     (lme, Y, L) float64 ...
    ...               ...
    Phi_11_0-200m    (lme, Y, L) float64 ...
    Phi_11_200-600m  (lme, Y, L) float64 ...
    Phi_11_0-600m    (lme, Y, L) float64 ...
    Phi_12_0-200m    (lme, Y, L) float64 ...
    Phi_12_200-600m  (lme, Y, L) float64 ...
    Phi_12_0-600m    (lme, Y, L) float64 ...

### select layer

In [5]:
#layer = '0-600m'
#layer = '0-200m'
layer = '200-600m'

In [6]:
import tools
alpha = 0.05
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
# lead year range: 1,2,...,9,10
ldyrs = range(1, 11)
# persistence ACC: var1; var2
# DPLE ACC:        var2; var3
lme = [1,2,3,4,5,6,7,8,9,10,65]

lmen = ['EBS','GoA','CC','GoC','GoM','SEUS','NEUS','SS','LN','IPH','AI']
# 9/13 phis
nphi = 13
#
corr_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); corr_p.mask=True
rcri_p = np.ma.zeros([nphi, len(lme), len(ldyrs)]); rcri_p.mask=True
corr_d = corr_p.copy()
rcri_d = rcri_p.copy()
corr13 = corr_p.copy()
rcri13 = rcri_p.copy()
nmae_p = corr_p.copy()
nmae_d = corr_p.copy()
#
for cc in range(nphi):
    for nn in range(len(lme)):
        box = lme[nn]
        var1 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948:yr2-1948+1].values
        #var1c = np.nanmean(ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1948:2018-1948].values)
        #var1 = var1 - var1c
        #
        for lead in range(1, 11):
            var2 = ds_hind[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1948+lead:yr2-1948+1+lead].values
            #var2c = np.nanmean()
            #var2 = var2 - var1c
            var3 = ds_fore[f'{var}_{cc}_{layer}'].sel(lme=box)[yr1-1954:yr2-1954+1, lead-1].values
            #var3c = np.nanmean(ds_fore[f'{var}_{cc}_{layer}'].sel(lme=box)[1954-1954:2018-1954, lead-1].values)
            #var3 = var3 - var3c
            # mask nan values
            mask1 = np.isnan(var1)
            mask2 = np.isnan(var2)
            mask3 = np.isnan(var3)
            mem1 = np.ma.array(var1, mask=mask1)
            mem2 = np.ma.array(var2, mask=mask2)
            mem3 = np.ma.array(var3, mask=mask3)
            # Persistence ACC & NMAE
            fmask = np.logical_and(mem1.mask, mem2.mask)
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen2 = mem2[~fmask]
                corr_p[cc, nn, lead-1] = pearsonr(nen1, nen2)[0]
                neff = tools.neff3(nen1, nen2, len(nen1))
                rcri_p[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                #rmse_p[cc, nn, lead-1] = np.sqrt(np.nanmean((nen1-nen2)**2))
                nmae_p[cc, nn, lead-1] = np.nanmean(np.abs(nen1-nen2)/np.nanstd(nen2))
            # DPLE ACC & NMAE
            fmask = np.logical_and(mem2.mask, mem3.mask)
            if np.sum(fmask) != len(fmask):
                nen2 = mem2[~fmask]
                nen3 = mem3[~fmask]
                corr_d[cc, nn, lead-1] = pearsonr(nen2, nen3)[0]
                neff = tools.neff3(nen2, nen3, len(nen2))
                rcri_d[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
                #
                #rmse_d[cc, nn, lead-1] = np.sqrt(np.nanmean((nen2-nen3)**2))
                nmae_d[cc, nn, lead-1] = np.nanmean(np.abs(nen3-nen2)/np.nanstd(nen2))
            # Correlation for ACC Difference Significance Test
            fmask = np.logical_and(mem1.mask, mem3.mask)
            if np.sum(fmask) != len(fmask):
                nen1 = mem1[~fmask]
                nen3 = mem3[~fmask]
                corr13[cc, nn, lead-1] = pearsonr(nen1, nen3)[0]
                neff = tools.neff3(nen1, nen3, len(nen1))
                rcri13[cc, nn, lead-1] = tools.calculate_parson_corr_critical_value(neff, alpha)
        #

In [7]:
data = xr.Dataset({"corr_p": (("nphi","nbox","nlead"),corr_p),\
                   "rcri_p": (("nphi","nbox","nlead"),rcri_p),\
                   "corr_d": (("nphi","nbox","nlead"),corr_d),\
                   "rcri_d": (("nphi","nbox","nlead"),rcri_d),\
                   "corr13": (("nphi","nbox","nlead"),corr13),\
                   "rcri13": (("nphi","nbox","nlead"),rcri13),\
                   "nmae_p": (("nphi","nbox","nlead"),nmae_p),\
                   "nmae_d": (("nphi","nbox","nlead"),nmae_d)})

In [8]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:  (nphi: 13, nbox: 11, nlead: 10)
Dimensions without coordinates: nphi, nbox, nlead
Data variables:
    corr_p   (nphi, nbox, nlead) float64 0.8682 0.6611 ... -0.09388 -0.05999
    rcri_p   (nphi, nbox, nlead) float64 0.7067 0.7067 0.7067 ... 0.8783 0.8783
    corr_d   (nphi, nbox, nlead) float64 0.8987 0.8289 0.7525 ... 0.5534 0.4855
    rcri_d   (nphi, nbox, nlead) float64 0.7067 0.7545 0.7545 ... 0.6664 0.576
    corr13   (nphi, nbox, nlead) float64 0.9377 0.852 0.757 ... -0.08898 -0.134
    rcri13   (nphi, nbox, nlead) float64 0.7067 0.7545 0.7545 ... 0.6664 0.6021
    nmae_p   (nphi, nbox, nlead) float64 0.4315 0.6691 0.8471 ... 1.222 1.183
    nmae_d   (nphi, nbox, nlead) float64 0.4002 0.4715 0.5376 ... 0.674 0.684

In [9]:
data.to_netcdf(f'{dout}/DPLE_LME_box01-11_ACC_NMAE_{var}_Eo_space_{layer}_leadyear_1-10.nc', mode='w')